In [2]:
from ResattUnet import *
from data import *
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'./train','image','label',data_gen_args,save_to_dir = None)
validGene= trainGenerator(2,'./validation','image','label',data_gen_args,save_to_dir = None)
# get u_net model
model = Attention_ResUNet()
model_checkpoint = ModelCheckpoint('ResattUnet.hdf5', monitor='loss',verbose=1, save_best_only=True)
hist = model.fit_generator(myGene, steps_per_epoch=1000, epochs=7, callbacks=[model_checkpoint], 
                     validation_data=validGene, validation_steps=3)


Epoch 1/7
Found 3 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Found 3 images belonging to 1 classes.
1000/1000 [==============================] - 6556s 7s/step - loss: 1.0222 - dice_coef: 0.6515 - val_loss: 3.6169 - val_dice_coef: 0.6399

Epoch 00001: loss improved from inf to 1.02215, saving model to unet_membrane.hdf5
Epoch 2/7
1000/1000 [==============================] - 5293s 5s/step - loss: 0.5492 - dice_coef: 0.8183 - val_loss: 0.7672 - val_dice_coef: 0.5951

Epoch 00002: loss improved from 1.02215 to 0.54919, saving model to unet_membrane.hdf5
Epoch 3/7
1000/1000 [==============================] - 5286s 5s/step - loss: 0.4524 - dice_coef: 0.8535 - val_loss: 0.4174 - val_dice_coef: 0.8452

Epoch 00003: loss improved from 0.54919 to 0.45241, saving model to unet_membrane.hdf5
Epoch 4/7
1000/1000 [==============================] - 5335s 5s/step - loss: 0.4154 - dice_coef: 0.8461 - val_loss: 0.8257 - val_dice_coef: 0

### Train with npy file

In [10]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [20]:
testGene = testGenerator('./test/image')
model = Attention_ResUNet()
model.load_weights("ResattUnet.hdf5")
results = model.predict_generator(testGene,3,verbose=1)
results[results>0.5]=1
results[results<=0.5]=0
saveResult("./test/results",results)

3/3 [==============================] - 10s 3s/step


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [21]:
img=io.imread('./test/label/0.png')
print(img)

[[255 255 255 ...   0   0   0]
 [255 255 255 ...   0   0   0]
 [255 255 255 ...   0   0   0]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [33]:
import matplotlib.pyplot as plt
img = io.imread('./test/label/0.png')
img = img.astype(np.double)
img1 = trans.resize(img,[256,256])

In [34]:
img = io.imread('./test/results/0_predict.png')
img=img.astype(np.double)
print(img)

[[255.   0.   0. ...   0.   0.   0.]
 [255.   0.   0. ...   0.   0.   0.]
 [255.   0.   0. ...   0.   0.   0.]
 ...
 [255.   0.   0. ...   0.   0.   0.]
 [255.   0.   0. ...   0.   0.   0.]
 [255.   0.   0. ...   0.   0. 255.]]


In [35]:
mean1=dice_coef(img1/255, img/255)
print("Mean Dice Coeff : ",mean1)

Mean Dice Coeff :  tf.Tensor(0.802546666678266, shape=(), dtype=float64)


In [37]:
img = io.imread('./test/label/1.png')
img = img.astype(np.double)
img2 = trans.resize(img,[256,256])
img = io.imread('./test/results/1_predict.png')
img=img.astype(np.double)
mean2=dice_coef(img2/255, img/255)
print("Mean Dice Coeff : ",mean2)

Mean Dice Coeff :  tf.Tensor(0.8346893623058487, shape=(), dtype=float64)


In [38]:
img = io.imread('./test/label/2.png')
img = img.astype(np.double)
img3 = trans.resize(img,[256,256])
img = io.imread('./test/results/2_predict.png')
img = img.astype(np.double)
mean3 = dice_coef(img3/255, img/255)
print("Mean Dice Coeff : ",mean3)

Mean Dice Coeff :  tf.Tensor(0.8238485005829229, shape=(), dtype=float64)


In [39]:
mean_average=(mean1+mean2+mean3)/3
print(mean_average)

tf.Tensor(0.8203615098556792, shape=(), dtype=float64)
